In [2]:
import pandas as pd

In [3]:
original_model = 'original'
trained_model = 'model_12'

original_df = pd.read_json('../logs/'+original_model+'/score.log', lines=True)
trained_df = pd.read_json('../logs/'+trained_model+'/score.log', lines=True)
original_fixed_df = pd.read_json('../logs/'+original_model+'/score_fixed.log', lines=True)
trained_fixed_df = pd.read_json('../logs/'+trained_model+'/score_fixed.log', lines=True)

size = min(len(trained_df), len(original_df))
fixed_size = min(len(trained_fixed_df), len(original_fixed_df))
original_df = original_df.head(size)
trained_df = trained_df.head(size)
original_fixed_df = original_fixed_df.head(fixed_size)
trained_fixed_df = trained_fixed_df.head(fixed_size)

# df['error_type'] = df['result'].apply(lambda x: x['error_type'])
# df['error'] = df['result'].apply(lambda x: x['error'])
# df['coverage'] = df['result'].apply(lambda x: x['coverage'])
# df['root_passed'] = df['result'].apply(lambda x: x['test_results']['root_passed'])
# df['root_failed'] = df['result'].apply(lambda x: x['test_results']['root_failed'])
# df['passed'] = df['result'].apply(lambda x: x['test_results']['passed'])
# df['failed'] = df['result'].apply(lambda x: x['test_results']['failed'])
# df['all_passed'] = df['result'].apply(lambda x: x['test_results']['all_passed'])

print('size', size)
print('fixed size', fixed_size)

for _, row in trained_fixed_df.head(1).iterrows():
    #print(row['completion'])
    print(row)
    print(type(row['result']), row['result'])

size 500
fixed size 500
hash                   3536598be3a14280f98fff5ab07e535dc21274d6166717...
project_path                   07be54bc22cf4dd2dd16a9788d2fc88952c438ae/
relative_go_package                                              wallet/
func_name                                             NewRecoveryManager
input_code             package wallet\n\nimport (\n\t"github.com/btcs...
prompt                 [{'content': '
You are an expert programmer. 
...
generate_time                                                  16.271075
completion             Here are some unit tests for the `NewRecoveryM...
score_time                                                     14.269766
result                 {'error': 'go mod tidy: code 1, 'strderr' go: ...
reward                                                               0.0
try_fix                                                             True
generate_time2                                                 26.612879
completion2            The 

In [5]:
original_reward = original_df['reward'].mean()
trained_reward = trained_df['reward'].mean()
original_fixed_reward = original_fixed_df['best_reward'].mean()
trained_fixed_reward = trained_fixed_df['best_reward'].mean()

def metrics(df1: pd.DataFrame, df2: pd.DataFrame, name: str):
    scores = []
    for df in [df1, df2]:
        df = df.copy()
        df['reward'] = df.apply(lambda x: float(x.get('best_reward', x.get('reward', None))), axis=1)
        df['coverage'] = df['result'].apply(lambda x: float(x['coverage']))
        df['mutation_score'] = df['result'].apply(lambda x: float(x['mutation_score']))
        df['error_type'] = df['result'].apply(lambda x: str(x['error_type']))
        df['all_passed'] = df['result'].apply(lambda x: int(x['test_results']['all_passed']))
        df['is_error'] = df['error_type'] != ''

        reward = df['reward'].mean()
        coverage = df['coverage'].mean()
        if coverage < 0.000001:
            coverage = 0.000001
        mutation_score = df['mutation_score'].mean()
        if mutation_score < 0.000001:
            mutation_score = 0.000001
        coverage_no_error = df.query('is_error == False')['coverage'].mean()
        # mutation_score_all_success = df.query('is_error == False')['mutation_score'].mean()
        mutation_score_all_success = df.query('all_passed == 1')['mutation_score'].mean()
        all_passed = len(df.query('all_passed == 1')) / len(df) * 100.0
        scores.append((reward, coverage, mutation_score, coverage_no_error, mutation_score_all_success, all_passed))


    reward1, coverage1, mutation_score1, coverage_no_error1, mutation_score_all_success1, all_passed1 = scores[0]
    reward2, coverage2, mutation_score2, coverage_no_error2, mutation_score_all_success2, all_passed2 = scores[1]

    return {
        'type': name,
        'count': len(df1),
        'original reward': '%.4f' % reward1,
        'trained reward': '%.4f' % reward2,
        'trained/original': '%.4f' % (reward2/reward1),
        'reward uplift, %': '%.2f' % (reward2/reward1*100 - 100),

        'original coverage': '%.4f' % coverage1,
        'trained coverage': '%.4f' % coverage2,
        'trained/original coverage': '%.4f' % (coverage2/coverage1),
        'coverage uplift, %': '%.2f' % (coverage2/coverage1*100 - 100),

        'original no error coverage': '%.4f' % coverage_no_error1,
        'trained no error coverage': '%.4f' % coverage_no_error2,
        'trained/original no error coverage': '%.4f' % (coverage_no_error2/coverage_no_error1),
        'coverage no error uplift, %': '%.2f' % (coverage_no_error2/coverage_no_error1*100 - 100),

        'original mutual': '%.4f' % mutation_score1,
        'trained mutual': '%.4f' % mutation_score2,
        'trained/original mutual': '%.4f' % (mutation_score2/mutation_score1),
        'mutual uplift, %': '%.2f' % (mutation_score2/mutation_score1*100 - 100),

        'original all passed mutual': '%.4f' % mutation_score_all_success1,
        'trained all passed mutual': '%.4f' % mutation_score_all_success2,
        'trained/original all passed mutual': '%.4f' % (mutation_score_all_success2/mutation_score_all_success1),
        'mutual all passed uplift, %': '%.2f' % (mutation_score_all_success2/mutation_score_all_success1*100 - 100),

        'original all passed': '%.4f' % all_passed1,
        'trained all passed': '%.4f' % all_passed2,
        'trained/original all passed': '%.4f' % (all_passed2/all_passed1),
        'all passed uplift, %': '%.2f' % (all_passed2/all_passed1*100 - 100),
    }

report = [
    metrics(original_df, trained_df, 'no fixing iteration'),
    metrics(original_fixed_df, trained_fixed_df, 'with fixing iteration'),
]

report_df = pd.DataFrame(data=report).set_index('type').transpose()

report_df.to_markdown('../logs/report.md')

report_df

type,no fixing iteration,with fixing iteration
count,500,500
original reward,0.1562,0.1585
trained reward,0.1818,0.1853
trained/original,1.1639,1.1688
"reward uplift, %",16.39,16.88
original coverage,15.0548,15.0548
trained coverage,18.3006,18.3006
trained/original coverage,1.2156,1.2156
"coverage uplift, %",21.56,21.56
original no error coverage,79.7879,79.7879
